https://wolfy.tistory.com/259#comment13236631

In [1]:
%run /content/drive/MyDrive/Colab\ Notebooks/VOCdataset_loader.ipynb

ERROR:root:File `'/content/drive/MyDrive/Colab Notebooks/VOCdataset_loader.ipynb.py'` not found.


In [2]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)

NameError: ignored

In [ ]:
import torch
import torch.nn as nn

In [ ]:
architecture_config = [
                       # (kernel_size, filters, stride, padding)
                       (7, 64, 2, 3),
                       "M",
                       (3, 192, 1, 1),
                       "M",
                       (1, 128, 1, 0),
                       (3, 256, 1, 1),
                       (1, 256, 1, 0),
                       (3, 512, 1, 1),
                       "M",
                       [(1, 256, 1, 0), (3, 512, 1, 1), 4],
                       (1, 512, 1, 0),
                       (3, 1024, 1, 1),
                       "M",
                       [(1,512,1,0), (3,1024,1,1), 2],
                       (3,1024, 1,1),
                       (3,1024, 2,1),
                       (3,1024, 1,1),
                       (3,1024, 1,1),
]

In [ ]:
class CNNBlock(nn.Module):
  def __init__(self, in_channels, out_channels, **kwargs):
    super(CNNBlock, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
    self.batchnorm = nn.BatchNorm2d(out_channels)
    self.leakyrelu = nn.LeakyReLU(0.1)

  def forward(self, x):
    return self.leakyrelu(self.batchnorm(self.conv(x)))    

In [ ]:
class YOLOv1(nn.Module):
  def __init__(self, in_channels=3, **kwargs):
    super(YOLOv1, self).__init__()
    self.architecture = architecture_config
    self.in_channels = in_channels
    self.darknet = self._create_conv_layers(self.architecture)
    self.fc = self._create_fc(**kwargs)
  
  def forward(self, x):
    x = self.darknet(x)
    print(x.shape)
    return self.fc(torch.flatten(x, 1))
  
  def _create_fc(self, split_size, num_boxes, num_classes):
    # S : split_size B : num_boxes, C : num_classes,
    S, B, C = split_size, num_boxes, num_classes

    return nn.Sequential(
      nn.Linear(1024 * S * S, 496),
      nn.Dropout(0.5),
      nn.LeakyReLU(0.1),
      nn.Linear(496, S * S * (C + B * 5)) # 30
    )

  def _create_conv_layers(self, architecture):
    layers = []
    in_channels = self.in_channels

    for x in architecture:
      # components are [conv2d, maxPool, repeated layers]
      if type(x) == tuple:
        kernel_size, out_channels, stride, padding = x
        layers.append(
          CNNBlock(
                in_channels, 
                out_channels,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding
          )
        )
        in_channels = out_channels
      if type(x) == str and x == 'M':
        layers.append(nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)))
      if type(x) == list:
        conv1 = x[0]
        conv2 = x[1]
        repeat_time = x[-1]
        for _ in range(repeat_time):
          layers += [
            CNNBlock(
                in_channels,
                conv1[1],
                kernel_size=conv1[0],
                stride=conv1[2],
                padding=conv1[3],
            )
          ]
          layers += [
            CNNBlock(
                conv1[1],
                conv2[1],
                kernel_size=conv2[0],
                stride=conv2[2],
                padding=conv2[3],
            )
          ]
          in_channels = conv2[1]
    return nn.Sequential(*layers)


In [ ]:
class YOLOv1Loss(nn.Module):
  def __init__(self, S=7, B=2, C=20):
    super(YOLOv1Loss, self).__init__()
    self.mse = nn.MSELoss(reduction="sum")
    self.S = S
    self.B = B
    self.C = C
    self.lambda_coord = 5.
    self.lambda_noobj = 0.5

  def intersection_over_union(pred, target):
    # pred.shape    : Batch_size, S, S, 4 (x, y, width, height)
    # target.shape  : Batch_size, S, S, 4
    pred_leftbot_x = pred[..., 0:1] - pred[..., 2:3]/2
    pred_leftbot_y = pred[..., 1:2] - pred[..., 3:4]/2
    pred_righttop_x = pred[...,0:1] + pred[..., 2:3]/2
    pred_righttop_y = pred[..., 1:2] + pred[..., 3:4]/2
    
    target_leftbot_x = target[..., 0:1] - target[..., 2:3]/2
    target_leftbot_y = target[..., 1:2] - target[..., 3:4]/2
    target_righttop_x = target[...,0:1] + target[..., 2:3]/2
    target_righttop_y = target[..., 1:2] + target[..., 3:4]/2

    x1 = torch.max(pred_leftbot_x, target_leftbot_x)
    y1 = torch.max(pred_leftbot_y, target_leftbot_y)
    x2 = torch.min(pred_righttop_x, target_righttop_x)
    y2 = torch.min(pred_righttop_y, target_righttop_y)
    
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

    pred_area = abs((pred_leftbot_x - pred_righttop_x) * (pred_leftbot_y - pred_righttop_y))
    target_area = abs((target_leftbot_x - target_righttop_x) * (target_leftbot_y - target_righttop_y))

    return intersection / (pred_area + target_area - intersection + 1e-6)


  def forward(self, predictions, targets):
    predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)
    # predictions.shape : Batch_size, S, S, 30
    # targets.shape     : Batch_size, S, S, 25
    pred_for_union1 = predictions[...,21:25]
    pred_for_union2 = predictions[...,26:30]
    target_for_union = targets[...,21:25]
    union1_result = YOLOv1Loss.intersection_over_union(pred_for_union1, target_for_union)
    union2_result = YOLOv1Loss.intersection_over_union(pred_for_union2, target_for_union)
    con_union = torch.concat(union1_result, union2_result, -1)
    _, argmax_ = torch.max(con_union, axis=-1)
    print(union1_result.shape, union2_result.shape)  
    return union1_result, union2_result


In [ ]:
model = YOLOv1(split_size=7, num_boxes=2, num_classes=20)

In [ ]:
iter_ = iter(train_dataloader)

In [ ]:
X, y = next(iter_)
pred = model.forward(X)

In [ ]:
loss = YOLOv1Loss()

In [ ]:
y.shape

In [ ]:
u1, u2 = loss(y, y)

In [ ]:
u1.unsqueeze(0).shape

In [ ]:
utot = torch.concat((u1, u2), -1)

In [ ]:
a,b = torch.max(utot, dim=-1)

In [ ]:
b.shape

In [ ]:
a = [98, 345, 420, 462]
height, width = 480, 640
a[0] /= width
a[1] /= height
a[2] /= width
a[3] /= height

In [ ]:
a

In [ ]:
a = [98, 345, 420, 462]
a_ = [(a[0] + a[2])/2, (a[1] + a[3])/2, a[2] - a[0], a[3] - a[1]]
a_

In [ ]:
height, width = 480, 640
a_[0] /= width
a_[1] /= height
a_[2] /= width
a_[3] /= height

In [ ]:
a_